# Project 3


## Executive Summary

More details througout the code below,  but this section summarizes our thinking on the key parts of our framework.


### Domain & Dataset

We examine a number of factors that relate to liquor store sales in the state of Iowa from January 1, 2015 to March 31, 2016.  The data set has 270,955 instances that consist of transaction-level data on 18 variables -- zip code, retail price, wholesale cost, product category, etc. 

Our impressions on the data set are based on the code and tabular analysis below:

### Solution Statement

Given store characteristics -- namely, number of products caried (a proxy for size of store), average retail price per bottle, number of stores in a zip code and the total size of liquor sales in a zip code, we can predict a store's slaes volumes using a regression model.  Given that much of the data revolves around local market characteristics (hence why we combine zip code level data with store data), our model will be useful in evaluating potential new store locations.  

We will be using different types of regression models including OLS, the Ridge Regression, the Lasso, and the Stochastic Gradient Descent.

### Metric

In order to assess the validity of our model, we will use R-squared, which is a measure of total variation of store sales volumes explained by our model.

### Benchmark

This is where domain level expertise in retail would help.  The glib answer is that we would like a model with as a high an R-squared as possible, but this would come at the risk of overfitting the model (reducing bias, but introducing variance, which we will try to control through parsimonious feature selection), which is of particular risk here as many of the data set variables are functions of one another (gross profit is a function of price).  We will try to balance introducing bias with A reasonable R2 benchmark would be above 0.80 with similar scores on the test and train portions of our dataset.


#### Importing Packages and Loading the Data

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline


# Overrides pandas default display option that limits number of columns displayed
pd.set_option("display.max_columns",999)
pd.set_option('precision',2)


In [2]:
df = pd.read_csv("Iowa_Liquor_sales_sample_10pct.csv")
df.head(5)

Date  Store Number         City Zip Code  County Number      County  \
0  11/04/2015          3717       SUMNER    50674            9.0      Bremer   
1  03/02/2016          2614    DAVENPORT    52807           82.0       Scott   
2  02/11/2016          2106  CEDAR FALLS    50613            7.0  Black Hawk   
3  02/03/2016          2501         AMES    50010           85.0       Story   
4  08/18/2015          3654      BELMOND    50421           99.0      Wright   

   Category              Category Name  Vendor Number  Item Number  \
0  1.05e+06           APRICOT BRANDIES             55        54436   
1  1.01e+06           BLENDED WHISKIES            395        27605   
2  1.01e+06  STRAIGHT BOURBON WHISKIES             65        19067   
3  1.07e+06         AMERICAN COCKTAILS            395        59154   
4  1.03e+06             VODKA 80 PROOF            297        35918   

            Item Description  Bottle Volume (ml) State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750             $4.50   
1                    Tin Cup                 750            $13.75   
2                   Jim Beam                1000            $12.59   
3    1800 Ultimate Margarita                1750             $9.50   
4         Five O'clock Vodka                1750             $7.20   

  State Bottle Retail  Bottles Sold Sale (Dollars)  Volume Sold (Liters)  \
0               $6.75            12         $81.00                   9.0   
1              $20.63             2         $41.26                   1.5   
2              $18.89            24        $453.36                  24.0   
3              $14.25             6         $85.50                  10.5   
4              $10.80            12        $129.60                  21.0   

   Volume Sold (Gallons)  
0                   2.38  
1                   0.40  
2                   6.34  
3                   2.77  
4                   5.55

In [3]:
df.shape

(270955, 18)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

In [5]:
# Code to get the number of unique values per column

header = list(df.columns.values)
unique = [(c, df[c].nunique()) for c in header]
unique

[('Date', 274),
 ('Store Number', 1400),
 ('City', 385),
 ('Zip Code', 415),
 ('County Number', 99),
 ('County', 99),
 ('Category', 83),
 ('Category Name', 71),
 ('Vendor Number', 116),
 ('Item Number', 2696),
 ('Item Description', 2173),
 ('Bottle Volume (ml)', 29),
 ('State Bottle Cost', 1086),
 ('State Bottle Retail', 1112),
 ('Bottles Sold', 137),
 ('Sale (Dollars)', 6580),
 ('Volume Sold (Liters)', 265),
 ('Volume Sold (Gallons)', 261)]

Start with a data set of 1400 stores - from the unique store numbers

#### Seeing what the data is like aggregated at the store/transaction level 

Store Number 2106 is in Des Moines

In [6]:
test_store2106 = df[df['Store Number']==2106]
test_store2106

Date  Store Number         City Zip Code  County Number  \
2       02/11/2016          2106  CEDAR FALLS    50613            7.0   
1161    08/20/2015          2106  CEDAR FALLS    50613            7.0   
1410    01/14/2016          2106  CEDAR FALLS    50613            7.0   
1525    03/12/2015          2106  CEDAR FALLS    50613            7.0   
1869    02/11/2016          2106  CEDAR FALLS    50613            7.0   
2348    07/30/2015          2106  CEDAR FALLS    50613            7.0   
2512    01/22/2015          2106  CEDAR FALLS    50613            7.0   
2663    08/13/2015          2106  CEDAR FALLS    50613            7.0   
3080    02/04/2016          2106  CEDAR FALLS    50613            7.0   
3298    03/03/2016          2106  CEDAR FALLS    50613            7.0   
3433    12/03/2015          2106  CEDAR FALLS    50613            7.0   
3827    01/07/2016          2106  CEDAR FALLS    50613            7.0   
4442    10/08/2015          2106  CEDAR FALLS    50613            7.0   
4814    10/29/2015          2106  CEDAR FALLS    50613            7.0   
4843    04/16/2015          2106  CEDAR FALLS    50613            7.0   
5511    09/03/2015          2106  CEDAR FALLS    50613            7.0   
5879    02/04/2016          2106  CEDAR FALLS    50613            7.0   
6065    05/07/2015          2106  CEDAR FALLS    50613            7.0   
7326    05/21/2015          2106  CEDAR FALLS    50613            7.0   
7469    02/12/2015          2106  CEDAR FALLS    50613            7.0   
8256    06/18/2015          2106  CEDAR FALLS    50613            7.0   
8741    08/27/2015          2106  CEDAR FALLS    50613            7.0   
8881    07/16/2015          2106  CEDAR FALLS    50613            7.0   
9057    04/09/2015          2106  CEDAR FALLS    50613            7.0   
9277    09/17/2015          2106  CEDAR FALLS    50613            7.0   
9334    07/09/2015          2106  CEDAR FALLS    50613            7.0   
9503    12/03/2015          2106  CEDAR FALLS    50613            7.0   
9943    03/24/2016          2106  CEDAR FALLS    50613            7.0   
10012   08/20/2015          2106  CEDAR FALLS    50613            7.0   
10232   08/27/2015          2106  CEDAR FALLS    50613            7.0   
...            ...           ...          ...      ...            ...   
257170  07/16/2015          2106  CEDAR FALLS    50613            7.0   
257182  10/22/2015          2106  CEDAR FALLS    50613            7.0   
257998  11/19/2015          2106  CEDAR FALLS    50613            7.0   
258145  01/29/2015          2106  CEDAR FALLS    50613            7.0   
258682  09/03/2015          2106  CEDAR FALLS    50613            7.0   
258867  12/10/2015          2106  CEDAR FALLS    50613            7.0   
259810  12/17/2015          2106  CEDAR FALLS    50613            7.0   
260397  10/22/2015          2106  CEDAR FALLS    50613            7.0   
261014  11/05/2015          2106  CEDAR FALLS    50613            7.0   
261146  04/09/2015          2106  CEDAR FALLS    50613            7.0   
262665  05/07/2015          2106  CEDAR FALLS    50613            7.0   
262951  03/03/2016          2106  CEDAR FALLS    50613            7.0   
263748  08/13/2015          2106  CEDAR FALLS    50613            7.0   
264276  02/26/2015          2106  CEDAR FALLS    50613            7.0   
265854  01/29/2015          2106  CEDAR FALLS    50613            7.0   
265989  02/19/2015          2106  CEDAR FALLS    50613            7.0   
266183  12/10/2015          2106  CEDAR FALLS    50613            7.0   
266293  03/19/2015          2106  CEDAR FALLS    50613            7.0   
267232  02/18/2016          2106  CEDAR FALLS    50613            7.0   
267319  02/26/2015          2106  CEDAR FALLS    50613            7.0   
267521  12/10/2015          2106  CEDAR FALLS    50613            7.0   
267683  02/26/2015          2106  CEDAR FALLS    50613            7.0   
268060  06/25/2015          2106  CEDAR FALLS    50613            7.0   

In [7]:
test_store2106.shape

(654, 18)

In [8]:
header = list(test_store2106.columns.values)
unique = [(c, test_store2106[c].nunique()) for c in header]
unique

[('Date', 65),
 ('Store Number', 1),
 ('City', 1),
 ('Zip Code', 1),
 ('County Number', 1),
 ('County', 1),
 ('Category', 52),
 ('Category Name', 50),
 ('Vendor Number', 33),
 ('Item Number', 281),
 ('Item Description', 248),
 ('Bottle Volume (ml)', 6),
 ('State Bottle Cost', 199),
 ('State Bottle Retail', 197),
 ('Bottles Sold', 19),
 ('Sale (Dollars)', 301),
 ('Volume Sold (Liters)', 32),
 ('Volume Sold (Gallons)', 32)]

In [9]:
test_store2106['Bottles Sold'].max()

180

In [10]:
test_store2106['Bottles Sold'].min()

1

## Cleaning the data

Task List:
* Remove "$" prices from characters and convert values to floats.
* Convert dates to pandas datetime objects
* Convert category floats to integers
* Drop or fill in bad values

##### Remove "$" prices from characters and convert values to floats

In [11]:
dollar_removal_cols = ["State Bottle Cost", "State Bottle Retail", "Sale (Dollars)"]

for col in dollar_removal_cols:
    df[col] = df[col].apply(lambda x: float(x[1:]))

In [12]:
df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0  11/04/2015          3717     SUMNER    50674            9.0  Bremer   
1  03/02/2016          2614  DAVENPORT    52807           82.0   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0  1.05e+06  APRICOT BRANDIES             55        54436   
1  1.01e+06  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   

   Volume Sold (Gallons)  
0                   2.38  
1                   0.40

##### Convert dates to pandas datetime objects

In [13]:
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")
df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0 2015-11-04          3717     SUMNER    50674            9.0  Bremer   
1 2016-03-02          2614  DAVENPORT    52807           82.0   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0  1.05e+06  APRICOT BRANDIES             55        54436   
1  1.01e+06  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   

   Volume Sold (Gallons)  
0                   2.38  
1                   0.40

In [14]:
df["Date"].max()

Timestamp('2016-03-31 00:00:00')

In [15]:
df["Date"].min()

Timestamp('2015-01-05 00:00:00')

##### Drop or replace 'bad' values

In [16]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269258 entries, 0 to 270954
Data columns (total 18 columns):
Date                     269258 non-null datetime64[ns]
Store Number             269258 non-null int64
City                     269258 non-null object
Zip Code                 269258 non-null object
County Number            269258 non-null float64
County                   269258 non-null object
Category                 269258 non-null float64
Category Name            269258 non-null object
Vendor Number            269258 non-null int64
Item Number              269258 non-null int64
Item Description         269258 non-null object
Bottle Volume (ml)       269258 non-null int64
State Bottle Cost        269258 non-null float64
State Bottle Retail      269258 non-null float64
Bottles Sold             269258 non-null int64
Sale (Dollars)           269258 non-null float64
Volume Sold (Liters)     269258 non-null float64
Volume Sold (Gallons)    269258 non-null float64
dtypes: datetim

In [17]:
# of stores after dropna drops from 1400 to 1378

df['Store Number'].nunique()

1378

In [18]:
df['Zip Code'].nunique()

412

##### Convert category floats to integers

In [19]:
df["County Number"] = df["County Number"].astype(int)
df["Category"] = df["Category"].astype(int)
df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0 2015-11-04          3717     SUMNER    50674              9  Bremer   
1 2016-03-02          2614  DAVENPORT    52807             82   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0   1051100  APRICOT BRANDIES             55        54436   
1   1011100  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   

   Volume Sold (Gallons)  
0                   2.38  
1                   0.40

### Simplifying the Product Categories for Later Analysis

In [20]:
def make_broad_category(x):
   x = x.lower()
   if 'liqueur' in x:
       return 'liqueur'
   elif 'whisk' in x or 'scotch' in x:
       return 'whiskey'
   elif 'vodka' in x:
       return 'vodka'
   elif 'gin' in x:
       return 'gin'
   elif 'tequila' in x:
       return 'tequila'
   elif 'schnapps' in x:
       return 'schnapps'
   elif 'rum' in x:
       return 'rum'
   elif 'brandies' in x:
       return 'brandy'
   elif 'creme' in x:
       return 'creme'
   elif 'beer' in x:
       return 'beer'
   elif 'amaretto' in x:
       return 'amaretto'
   elif 'anisette' in x:
       return 'anisette'
   else:
       return 'misc'
   


In [21]:
df['Broad Category'] = df['Category Name'].map(make_broad_category)

In [22]:
df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0 2015-11-04          3717     SUMNER    50674              9  Bremer   
1 2016-03-02          2614  DAVENPORT    52807             82   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0   1051100  APRICOT BRANDIES             55        54436   
1   1011100  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   

   Volume Sold (Gallons) Broad Category  
0                   2.38         brandy  
1                   0.40        whiskey

### At this point now that everything has been converted to numbers - taking a look at outliers

In [23]:
df.describe()

Store Number  County Number  Category  Vendor Number  Item Number  \
count     269258.00      269258.00  2.69e+05      269258.00    269258.00   
mean        3587.70          57.23  1.04e+06         256.36     45944.81   
std          946.51          27.34  5.02e+04         140.98     52557.19   
min         2106.00           1.00  1.01e+06          10.00       168.00   
25%         2604.00          31.00  1.01e+06         115.00     26827.00   
50%         3721.00          62.00  1.03e+06         260.00     38176.00   
75%         4376.00          77.00  1.06e+06         380.00     64573.00   
max         9023.00          99.00  1.70e+06         978.00    995507.00   

       Bottle Volume (ml)  State Bottle Cost  State Bottle Retail  \
count           269258.00          269258.00            269258.00   
mean               925.22               9.76                14.66   
std                492.84               7.04                10.56   
min                 50.00               0.89                 1.34   
25%                750.00               5.50                 8.25   
50%                750.00               8.00                12.11   
75%               1000.00              11.92                17.88   
max               6000.00             425.00               637.50   

       Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
count     269258.00       269258.00             269258.00   
mean           9.87          128.83                  8.99   
std           24.07          383.48                 28.96   
min            1.00            1.34                  0.10   
25%            2.00           30.42                  1.50   
50%            6.00           70.56                  5.25   
75%           12.00          135.00                 10.50   
max         2508.00        36392.40               2508.00   

       Volume Sold (Gallons)  
count              269258.00  
mean                    2.38  
std                     7.65  
min                     0.03  
25%                     0.40  
50%                     1.39  
75%                     2.77  
max                   662.54

In [24]:
df.shape

(269258, 19)

In [25]:
# Top percentile of bottle sales

df['Bottles Sold'].quantile(.99)

60.0

In [26]:
wholesale_df = df[df['Bottles Sold'] >= 75]

In [27]:
wholesale_df.shape # of of transactions that involved > 85 bottles sold -- probably indicates a wholesaler than retailer

(2101, 19)

In [28]:
# how to calculate number of observations eliminated if you limit transaction sizes to x percentile

pct_of_total_obs_elim = (wholesale_df.shape[0]*1.0 / df.shape[0])*100
pct_of_total_obs_elim

0.780292507557807

In [29]:
df_less_ws = df[df['Bottles Sold'] <= 75]

In [30]:
df_less_ws.shape

(267158, 19)

In [31]:
df_less_ws.describe()

Store Number  County Number  Category  Vendor Number  Item Number  \
count     267158.00      267158.00  2.67e+05      267158.00    267158.00   
mean        3587.76          57.19  1.04e+06         256.35     45925.21   
std          946.07          27.35  5.00e+04         140.98     52326.14   
min         2106.00           1.00  1.01e+06          10.00       168.00   
25%         2604.00          31.00  1.01e+06         115.00     26827.00   
50%         3721.00          62.00  1.03e+06         260.00     38176.00   
75%         4376.00          77.00  1.06e+06         380.00     64573.00   
max         9023.00          99.00  1.70e+06         978.00    995507.00   

       Bottle Volume (ml)  State Bottle Cost  State Bottle Retail  \
count           267158.00          267158.00            267158.00   
mean               924.28               9.76                14.66   
std                492.51               7.05                10.57   
min                 50.00               0.89                 1.34   
25%                750.00               5.50                 8.27   
50%                750.00               8.00                12.00   
75%               1000.00              11.88                17.82   
max               6000.00             425.00               637.50   

       Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
count     267158.00       267158.00             267158.00   
mean           8.44          107.81                  7.39   
std            9.18          141.54                  8.79   
min            1.00            1.34                  0.10   
25%            2.00           30.06                  1.50   
50%            6.00           69.60                  5.25   
75%           12.00          134.88                 10.50   
max           75.00         3712.50                153.00   

       Volume Sold (Gallons)  
count              267158.00  
mean                    1.95  
std                     2.32  
min                     0.03  
25%                     0.40  
50%                     1.39  
75%                     2.77  
max                    40.42

In [32]:
# Top percentile of retail price

df['State Bottle Retail'].quantile(.999)

98.989999999999995

In [33]:
topshelf_df =  df[df['State Bottle Retail'] >= 100]
topshelf_df.shape

(220, 19)

In [34]:
pct_of_total_obs_elim = (topshelf_df.shape[0]*1.0 / df.shape[0])*100
pct_of_total_obs_elim

0.08170602173380177

### Taking Out the Outliers on Wholesale and Top Shelf

In [35]:
df = df[df['Bottles Sold'] <= 75]

In [36]:
df.shape

(267158, 19)

In [37]:
df = df[df['State Bottle Retail'] <= 100]

In [38]:
df.shape

(266938, 19)

In [39]:
df.describe()

Store Number  County Number  Category  Vendor Number  Item Number  \
count     266938.00      266938.00  2.67e+05      266938.00     266938.0   
mean        3587.98          57.19  1.04e+06         256.31      45735.0   
std          946.02          27.35  5.00e+04         141.00      50561.2   
min         2106.00           1.00  1.01e+06          10.00        168.0   
25%         2604.00          31.00  1.01e+06         115.00      26827.0   
50%         3721.00          62.00  1.03e+06         260.00      38176.0   
75%         4377.00          77.00  1.06e+06         380.00      64573.0   
max         9023.00          99.00  1.70e+06         978.00     994945.0   

       Bottle Volume (ml)  State Bottle Cost  State Bottle Retail  \
count           266938.00          266938.00            266938.00   
mean               924.21               9.68                14.53   
std                491.78               6.29                 9.43   
min                 50.00               0.89                 1.34   
25%                750.00               5.50                 8.25   
50%                750.00               8.00                12.00   
75%               1000.00              11.83                17.75   
max               4800.00              65.99                98.99   

       Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
count     266938.00       266938.00             266938.00   
mean           8.44          107.59                  7.39   
std            9.18          140.61                  8.79   
min            1.00            1.34                  0.10   
25%            2.00           30.04                  1.50   
50%            6.00           69.57                  5.25   
75%           12.00          134.88                 10.50   
max           75.00         3429.36                153.00   

       Volume Sold (Gallons)  
count              266938.00  
mean                    1.95  
std                     2.32  
min                     0.03  
25%                     0.40  
50%                     1.39  
75%                     2.77  
max                    40.42

In [40]:
# of stores after eliminating outliers - remains virtually the same - 1378 to 1377

df['Store Number'].nunique()

1377

## Filter the Data

Some stores may have opened or closed in 2015. These data points will heavily skew our models, so we need to filter them out or find a way to deal with them.

You'll need to provide a summary in your project report about these data points. You may also consider using the monthly sales in your model and including other information (number of months or days each store is open) in your data to handle these unusual cases.

Let's record the first and last sales dates for each store. We'll save this information for later when we fit our models.

##### Determine which stores were open all of 2015 - finding the first and last sale date

In [41]:
dates = df.groupby(by=["Store Number"], as_index=False)
dates = dates.agg({"Date": [np.min, np.max]})

##### For formatting purposes, this strips the empty cell above the first row

In [42]:
dates.columns = [' '.join(col).strip() for col in dates.columns.values]
dates.head()

Store Number  Date amin  Date amax
0          2106 2015-01-08 2016-03-31
1          2113 2015-01-07 2016-03-23
2          2130 2015-01-08 2016-03-31
3          2152 2015-01-08 2016-02-25
4          2178 2015-01-07 2016-03-30

##### This code filters out stores that opened or closed during 2015 using a mask

Stores that opened after March 1st and stores that closed before October 1 are to be considered partial year stores

In [43]:
lower_cutoff = pd.Timestamp("20150301")
upper_cutoff = pd.Timestamp("20151001")

mask = (dates['Date amin'] < lower_cutoff) & (dates['Date amax'] > upper_cutoff)

good_stores = dates[mask]["Store Number"]

In [44]:
good_stores.nunique()

1161

In [45]:
# defining the new DataFrame

good_stores_df = df[df["Store Number"].isin(good_stores)]

In [46]:
good_stores_df.head()

Date  Store Number         City Zip Code  County Number      County  \
0 2015-11-04          3717       SUMNER    50674              9      Bremer   
1 2016-03-02          2614    DAVENPORT    52807             82       Scott   
2 2016-02-11          2106  CEDAR FALLS    50613              7  Black Hawk   
3 2016-02-03          2501         AMES    50010             85       Story   
4 2015-08-18          3654      BELMOND    50421             99      Wright   

   Category              Category Name  Vendor Number  Item Number  \
0   1051100           APRICOT BRANDIES             55        54436   
1   1011100           BLENDED WHISKIES            395        27605   
2   1011200  STRAIGHT BOURBON WHISKIES             65        19067   
3   1071100         AMERICAN COCKTAILS            395        59154   
4   1031080             VODKA 80 PROOF            297        35918   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   
2                   Jim Beam                1000              12.59   
3    1800 Ultimate Margarita                1750               9.50   
4         Five O'clock Vodka                1750               7.20   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   
2                18.89            24          453.36                  24.0   
3                14.25             6           85.50                  10.5   
4                10.80            12          129.60                  21.0   

   Volume Sold (Gallons) Broad Category  
0                   2.38         brandy  
1                   0.40        whiskey  
2                   6.34        whiskey  
3                   2.77           misc  
4                   5.55          vodka

In [47]:
# After removal of partial year stores get to 1161

good_stores_df['Store Number'].nunique()

1161

In [48]:
good_stores_df.shape # taking out the partial year stores - lose 5% of observations - acceptable adjustment

(253937, 19)

In [49]:
df.shape

(266938, 19)

In [50]:
pct_of_total_obs_elim = (1 - (good_stores_df.shape[0]*1.0 / df.shape[0])) * 100
pct_of_total_obs_elim

4.870419348313093

Taking out the partial year stores would result in losing in less than 5% of the total observations -- seems reasonable

In [51]:
df = good_stores_df
df.describe()

Store Number  County Number  Category  Vendor Number  Item Number  \
count     253937.00      253937.00  2.54e+05      253937.00    253937.00   
mean        3525.04          57.02  1.04e+06         256.33     45884.95   
std          914.30          27.47  4.99e+04         141.33     51461.01   
min         2106.00           1.00  1.01e+06          10.00       168.00   
25%         2597.00          31.00  1.01e+06         115.00     26827.00   
50%         3666.00          61.00  1.03e+06         260.00     38176.00   
75%         4296.00          77.00  1.06e+06         380.00     64676.00   
max         9010.00          99.00  1.70e+06         978.00    994945.00   

       Bottle Volume (ml)  State Bottle Cost  State Bottle Retail  \
count           253937.00          253937.00            253937.00   
mean               927.53               9.71                14.58   
std                490.01               6.29                 9.44   
min                 50.00               0.89                 1.34   
25%                750.00               5.51                 8.31   
50%                750.00               8.00                12.30   
75%               1000.00              11.92                17.88   
max               4800.00              65.99                98.99   

       Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
count     253937.00       253937.00             253937.00   
mean           8.43          108.24                  7.44   
std            9.16          141.76                  8.87   
min            1.00            1.34                  0.10   
25%            2.00           30.02                  1.50   
50%            6.00           69.60                  5.25   
75%           12.00          134.97                 10.50   
max           75.00         3429.36                153.00   

       Volume Sold (Gallons)  
count              253937.00  
mean                    1.97  
std                     2.34  
min                     0.03  
25%                     0.40  
50%                     1.39  
75%                     2.77  
max                    40.42

## Computing New Columns and Tables

Since we're trying to predict sales and/or profits, we'll want to compute some intermediate data. There are a lot of ways to do this and good use of pandas is crucial. For example, for each transaction we may want to know:
* unit margin = retail cost minus bottle cost
* gross margin = gross margin * bottles sold
* price per bottle
* price per liter

We'll need to make a new dataframe that indexes quantities by store:
* sales per store for all of 2015
* total volumes sold
* mean transaction revenue, gross margin, price per bottle, price per liter, etc.
* average sales per day
* number of days open (count of sales per store) - this is wrong because there would be more than 365!!!

Make sure to retain other variables that we'll want to use to build our models, such as zip code, county number, city, etc. We recommend that you spend some time thinking about the model you may want to fit and computing enough of the suggested quantities to give you a few options.

Bonus tasks:
* Restrict your attention to stores that were open for all of 2015 and Q1 2016. Stores that opened or closed in 2015 will introduce outliers into your data.
* For each transaction we have the item category. You may be able to determine the store type (primarily wine, liquor, all types of alcohol, etc.) by the most common transaction category for each store. This could be a useful categorical variable for modelling. 

### Transaction Level Metrics

##### Calculating Unit Margin

In [52]:
df["Transaction Unit Margin"] = (df["State Bottle Retail"] - df["State Bottle Cost"])

##### Calculating Gross Margin - Perhaps Name this Gross Profits

In [53]:
df["Gross Profit (Dollars)"] = (df["State Bottle Retail"] - df["State Bottle Cost"]) * df["Bottles Sold"]

##### Calculating Gross Margin %

In [54]:
df["Transaction Unit Margin%"] = (1 - (df["State Bottle Cost"]/df["State Bottle Retail"]))*100

##### Revenue per Liter

In [55]:
df["Transaction Revenue per Liter"] = df["Sale (Dollars)"] / df["Volume Sold (Liters)"]

##### Gross Margin per Liter

In [56]:
df["Transaction Gross Margin per Liter"] = (((df["State Bottle Retail"] - df["State Bottle Cost"]) 
                                             * df["Bottles Sold"]) / df["Volume Sold (Liters)"])

Checking our DataFrame after new column additions

In [57]:
df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0 2015-11-04          3717     SUMNER    50674              9  Bremer   
1 2016-03-02          2614  DAVENPORT    52807             82   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0   1051100  APRICOT BRANDIES             55        54436   
1   1011100  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  State Bottle Cost  \
0  Mr. Boston Apricot Brandy                 750               4.50   
1                    Tin Cup                 750              13.75   

   State Bottle Retail  Bottles Sold  Sale (Dollars)  Volume Sold (Liters)  \
0                 6.75            12           81.00                   9.0   
1                20.63             2           41.26                   1.5   

   Volume Sold (Gallons) Broad Category  Transaction Unit Margin  \
0                   2.38         brandy                     2.25   
1                   0.40        whiskey                     6.88   

   Gross Profit (Dollars)  Transaction Unit Margin%  \
0                   27.00                     33.33   
1                   13.76                     33.35   

   Transaction Revenue per Liter  Transaction Gross Margin per Liter  
0                           9.00                                3.00  
1                          27.51                                9.17

In [58]:
df = df.rename(columns={'State Bottle Cost': 'Cost per Bottle',
                        'State Bottle Retail': 'Retail Price per Bottle',
                        'Sale (Dollars)': 'Revenues (Dollars)'
                       })

df.head(2)

Date  Store Number       City Zip Code  County Number  County  \
0 2015-11-04          3717     SUMNER    50674              9  Bremer   
1 2016-03-02          2614  DAVENPORT    52807             82   Scott   

   Category     Category Name  Vendor Number  Item Number  \
0   1051100  APRICOT BRANDIES             55        54436   
1   1011100  BLENDED WHISKIES            395        27605   

            Item Description  Bottle Volume (ml)  Cost per Bottle  \
0  Mr. Boston Apricot Brandy                 750             4.50   
1                    Tin Cup                 750            13.75   

   Retail Price per Bottle  Bottles Sold  Revenues (Dollars)  \
0                     6.75            12               81.00   
1                    20.63             2               41.26   

   Volume Sold (Liters)  Volume Sold (Gallons) Broad Category  \
0                   9.0                   2.38         brandy   
1                   1.5                   0.40        whiskey   

   Transaction Unit Margin  Gross Profit (Dollars)  Transaction Unit Margin%  \
0                     2.25                   27.00                     33.33   
1                     6.88                   13.76                     33.35   

   Transaction Revenue per Liter  Transaction Gross Margin per Liter  
0                           9.00                                3.00  
1                          27.51                                9.17

### Calculating per Store Metrics for 2015

Step 1 - Filtering by start and end dates (January 1, 2015 and December 31, 2015, respectively)

In [59]:
df.sort_values(by=["Store Number", "Date"], inplace=True)

start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")

mask_2015 = (df['Date'] >= start_date) & (df['Date'] <= end_date)
stores_2015_df = df[mask_2015]

In [60]:
stores_2015_df.shape

(206344, 24)

In [61]:
# Calculating total Iowa liquor sales 2015
iowa_liquor_sales_2015 = stores_2015_df['Revenues (Dollars)'].sum()
iowa_liquor_sales_2015

22505492.67000281

In [62]:
# Calculating total Iowa gross margin 2015
iowa_liquor_gross_margin_2015 = stores_2015_df['Gross Profit (Dollars)'].sum()
iowa_liquor_gross_margin_2015

7520830.2599976845

##### Calculating Total Iowa Sales for 1Q'2015

Goal of this code block is to generate a dataframe that has info for 1Q'15, which we will then use to combine with data from question is to see if there is any seasonality in sales.  Our original data set runs from January 1, 2015 to March 31, 2016.  If sales were significantly skewed higher or lower in 1Q, by including the whole data set we would be skewing our numbers as we would have two 1Q's (2015 and 2016).

In [63]:
df.sort_values(by=["Store Number", "Date"], inplace=True)

start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20150331")

mask_1Q15 = (df['Date'] >= start_date) & (df['Date'] <= end_date)
stores_1Q15_df = df[mask_1Q15]

In [64]:
stores_1Q15_df.shape

(47432, 24)

In [65]:
# Calculating total Iowa liquor sales 1Q15
iowa_liquor_sales_1Q15 = stores_1Q15_df['Revenues (Dollars)'].sum()
iowa_liquor_sales_1Q15

5014523.259999812

In [66]:
iowa_liquor_sales_2015/4*1.0

5626373.167500703

In [67]:
iowa_liquor_sales_1Q15/iowa_liquor_sales_2015 # expect this to be close to 0.25 if no seasonality

0.22281330755685186

In [68]:
# Calculating total Iowa gross margin 2015
iowa_liquor_gross_margin_1Q15 = stores_1Q15_df['Gross Profit (Dollars)'].sum()
iowa_liquor_gross_margin_1Q15

1675757.2200000596

In [69]:
iowa_liquor_gross_margin_2015/4*1.0

1880207.5649994211

In [70]:
iowa_liquor_gross_margin_1Q15/iowa_liquor_gross_margin_2015

0.22281545548410975

##### Conclusion: no real seasonality in sales and gross margin

##### Looking at Q12016 Sales - Was There Any Growth Year Over Year?

In [71]:
df.sort_values(by=["Store Number", "Date"], inplace=True)

start_date = pd.Timestamp("20160101")
end_date = pd.Timestamp("20160331")

mask_1Q16 = (df['Date'] >= start_date) & (df['Date'] <= end_date)
stores_1Q16_df = df[mask_1Q16]

In [72]:
stores_1Q16_df.shape

(47593, 24)

In [73]:
# Calculating total Iowa liquor sales 1Q16
iowa_liquor_sales_1Q16 = stores_1Q16_df['Revenues (Dollars)'].sum()
iowa_liquor_sales_1Q16

4979727.799999831

In [74]:
((iowa_liquor_sales_1Q16/iowa_liquor_sales_1Q15)-1)*100

-0.6938936803332685

In [75]:
# System wide year-over-year sales growth was flat

### Back to Calculating Sales per Store for 2015

Step 2 - Group by Store Number

In [76]:
stores_2015_df = stores_2015_df.groupby(by="Store Number", as_index=False)

### Step 3 - Compute sums, means for key metrics and collapse the column indicies

In [77]:
# When you see np.mean calculated for a column -- interpret it as average per store

stores_2015_df = stores_2015_df.agg({"Bottles Sold": [np.sum, np.mean, 'max', 'min'], 
                                     "Volume Sold (Liters)": [np.sum, np.mean],
        
                                     "Revenues (Dollars)": [np.sum, np.mean, 'count'],
                                     "Gross Profit (Dollars)": [np.sum, np.mean],
                                     
                                     "Retail Price per Bottle": np.mean,
                                     "Cost per Bottle": np.mean,
                                     "Transaction Unit Margin": np.mean,
                                     "Transaction Unit Margin%": np.mean,
                                     
                                     "Transaction Revenue per Liter": np.mean,
                                     "Transaction Gross Margin per Liter": np.mean,
                                     
                                     "Zip Code": lambda x: x.iloc[0], # just extract once, should be the same
                                     "City": lambda x: x.iloc[0],
                                     "County Number": lambda x: x.iloc[0],
                                     "County": lambda x: x.iloc[0],
                                     "Category": 'nunique',
                                     "Broad Category": lambda x:x.value_counts().index[0], # returns the most common
                                     "Vendor Number": 'nunique',
                                     "Item Number": 'nunique',
                                   
                                     "Date": ['nunique', 'count'], 
                                     # confirming the number of transactions per store in time period
                                     # count should match count for Sale (Dollars), but 'nunique' = number of days open
                                   
                                    })

stores_2015_df.columns = [' '.join(col).strip() for col in stores_2015_df.columns.values]

stores_2015_df.head(2)

Store Number  Category nunique  Cost per Bottle mean  \
0          2106                48                 10.18   
1          2113                36                 10.87   

   Gross Profit (Dollars) sum  Gross Profit (Dollars) mean  \
0                    41723.72                        81.65   
1                     3109.04                        21.15   

   Transaction Unit Margin mean  Transaction Gross Margin per Liter mean  \
0                          5.10                                     5.89   
1                          5.45                                     6.17   

  City <lambda>  Vendor Number nunique  Transaction Unit Margin% mean  \
0   CEDAR FALLS                     32                          33.40   
1        GOWRIE                     23                          33.39   

   Revenues (Dollars) sum  Revenues (Dollars) mean  Revenues (Dollars) count  \
0               125057.54                   244.73                       511   
1                 9310.22                    63.33                       147   

  Zip Code <lambda> Broad Category <lambda>  Retail Price per Bottle mean  \
0             50613                 whiskey                         15.29   
1             50543                 whiskey                         16.32   

  County <lambda>  County Number <lambda>  Bottles Sold sum  \
0      Black Hawk                       7              8625   
1         Webster                      94               671   

   Bottles Sold mean  Bottles Sold max  Bottles Sold min  \
0              16.88                72                 1   
1               4.56                24                 1   

   Volume Sold (Liters) sum  Volume Sold (Liters) mean  Date nunique  \
0                   8020.35                      15.70            52   
1                    659.85                       4.49            47   

   Date count  Item Number nunique  Transaction Revenue per Liter mean  
0         511                  244                               17.63  
1         147                   94                               18.51

##### Renaming column names

In [78]:
stores_2015_df.columns = stores_2015_df.columns.str.replace(' <lambda>', '')

In [79]:
stores_2015_df.drop(['Date count'], axis=1, inplace=True)

In [80]:
stores_2015_df.head(2)

Store Number  Category nunique  Cost per Bottle mean  \
0          2106                48                 10.18   
1          2113                36                 10.87   

   Gross Profit (Dollars) sum  Gross Profit (Dollars) mean  \
0                    41723.72                        81.65   
1                     3109.04                        21.15   

   Transaction Unit Margin mean  Transaction Gross Margin per Liter mean  \
0                          5.10                                     5.89   
1                          5.45                                     6.17   

          City  Vendor Number nunique  Transaction Unit Margin% mean  \
0  CEDAR FALLS                     32                          33.40   
1       GOWRIE                     23                          33.39   

   Revenues (Dollars) sum  Revenues (Dollars) mean  Revenues (Dollars) count  \
0               125057.54                   244.73                       511   
1                 9310.22                    63.33                       147   

  Zip Code Broad Category  Retail Price per Bottle mean      County  \
0    50613        whiskey                         15.29  Black Hawk   
1    50543        whiskey                         16.32     Webster   

   County Number  Bottles Sold sum  Bottles Sold mean  Bottles Sold max  \
0              7              8625              16.88                72   
1             94               671               4.56                24   

   Bottles Sold min  Volume Sold (Liters) sum  Volume Sold (Liters) mean  \
0                 1                   8020.35                      15.70   
1                 1                    659.85                       4.49   

   Date nunique  Item Number nunique  Transaction Revenue per Liter mean  
0            52                  244                               17.63  
1            47                   94                               18.51

In [81]:
stores_2015_df.rename(columns={'Category nunique': 'Number of Liquor Categories Offered',
                               'Category Name': 'Best Selling Category',
                               'Broad Category': 'Best Selling Broad Category',
                               'Cost per Bottle mean': 'Average Transaction Cost per Bottle',
                               'Gross Profit (Dollars) sum': 'Store Gross Profit (Dollars)',
                               'Gross Profit (Dollars) mean': 'Average Transaction Gross Profit',
                               'Transaction Unit Margin mean': 'Average Transaction Unit Margin',
                               'Vendor Number nunique': 'Number of Suppliers',
                               'Transaction Unit Margin% mean': 'Average Transaction Unit Margin%',
                               'Revenues (Dollars) sum': 'Store Revenues (Dollars)',
                               'Revenues (Dollars) mean': 'Average Revenues per Transaction',
                               'Revenues (Dollars) count': 'Number of Transactions in Period',
                               'Retail Price per Bottle mean': 'Average Retail (Transaction) Price per Bottle',
                               'Bottles Sold sum': 'Store Bottles Sold',
                               'Bottles Sold mean': 'Average Bottles Sold per Transaction',
                               'Bottles Sold max': 'Max Bottles Sold per Transaction',
                               'Bottles Sold min': 'Min Bottles Sold per Transaction',
                               'Volume Sold (Liters) sum': 'Store Volume (liters)',
                               'Volume Sold (Liters) mean': 'Average Liters Sold per Transaction',
                               'Date nunique': 'Store Operating Days',
                               'Item Number nunique': 'Number of Products Offered',
                               'Transaction Revenue per Liter mean': 'Average Transaction Revenue per Liter'
                              }, inplace=True)

stores_2015_df.head(2)

Store Number  Number of Liquor Categories Offered  \
0          2106                                   48   
1          2113                                   36   

   Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
0                                10.18                      41723.72   
1                                10.87                       3109.04   

   Average Transaction Gross Profit  Average Transaction Unit Margin  \
0                             81.65                             5.10   
1                             21.15                             5.45   

   Transaction Gross Margin per Liter mean         City  Number of Suppliers  \
0                                     5.89  CEDAR FALLS                   32   
1                                     6.17       GOWRIE                   23   

   Average Transaction Unit Margin%  Store Revenues (Dollars)  \
0                             33.40                 125057.54   
1                             33.39                   9310.22   

   Average Revenues per Transaction  Number of Transactions in Period  \
0                            244.73                               511   
1                             63.33                               147   

  Zip Code Best Selling Broad Category  \
0    50613                     whiskey   
1    50543                     whiskey   

   Average Retail (Transaction) Price per Bottle      County  County Number  \
0                                          15.29  Black Hawk              7   
1                                          16.32     Webster             94   

   Store Bottles Sold  Average Bottles Sold per Transaction  \
0                8625                                 16.88   
1                 671                                  4.56   

   Max Bottles Sold per Transaction  Min Bottles Sold per Transaction  \
0                                72                                 1   
1                                24                                 1   

   Store Volume (liters)  Average Liters Sold per Transaction  \
0                8020.35                                15.70   
1                 659.85                                 4.49   

   Store Operating Days  Number of Products Offered  \
0                    52                         244   
1                    47                          94   

   Average Transaction Revenue per Liter  
0                                  17.63  
1                                  18.51

In [82]:
stores_2015_df['Store Revenues (Dollars)'].sum() # total sales for all of the stores in the dataset in 2015

22505492.670000024

### Now we can create store level metrics

In [83]:
stores_2015_df['Store Bottles Sold per Day'] = (stores_2015_df['Store Bottles Sold'] / 
                                                stores_2015_df['Store Operating Days']) 

In [84]:
stores_2015_df['Store Liters Sold per Day'] = (stores_2015_df['Store Volume (liters)'] / 
                                                stores_2015_df['Store Operating Days']) 

In [85]:
stores_2015_df['Store Revenues per Day'] = (stores_2015_df['Store Revenues (Dollars)'] / 
                                            stores_2015_df['Store Operating Days']) 

In [86]:
stores_2015_df['Store Revenues per Bottle'] = (stores_2015_df['Store Revenues (Dollars)'] / 
                                               stores_2015_df['Store Bottles Sold'])

In [87]:
stores_2015_df['Store Revenues per Liter'] = (stores_2015_df['Store Revenues (Dollars)'] / 
                                               stores_2015_df['Store Volume (liters)'])

In [88]:
stores_2015_df['Store Gross Profits per Day'] = (stores_2015_df['Store Gross Profit (Dollars)'] / 
                                                 stores_2015_df['Store Operating Days']) 

In [89]:
stores_2015_df['Store Gross Profit Margin%'] = (stores_2015_df['Store Gross Profit (Dollars)'] / 
                                                stores_2015_df['Store Revenues (Dollars)']) * 100

In [90]:
stores_2015_df['Store Unit Margin'] = (stores_2015_df['Store Gross Profit (Dollars)'] / 
                                       stores_2015_df['Store Bottles Sold']) 

In [91]:
stores_2015_df['Store Gross Profit per Liter'] = (stores_2015_df['Store Gross Profit (Dollars)'] / 
                                                  stores_2015_df['Store Volume (liters)'])

In [92]:
stores_2015_df['Transactions per Day'] = (stores_2015_df['Number of Transactions in Period'] / 
                                                  stores_2015_df['Store Operating Days'])

In [93]:
stores_2015_df.head(2)

Store Number  Number of Liquor Categories Offered  \
0          2106                                   48   
1          2113                                   36   

   Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
0                                10.18                      41723.72   
1                                10.87                       3109.04   

   Average Transaction Gross Profit  Average Transaction Unit Margin  \
0                             81.65                             5.10   
1                             21.15                             5.45   

   Transaction Gross Margin per Liter mean         City  Number of Suppliers  \
0                                     5.89  CEDAR FALLS                   32   
1                                     6.17       GOWRIE                   23   

   Average Transaction Unit Margin%  Store Revenues (Dollars)  \
0                             33.40                 125057.54   
1                             33.39                   9310.22   

   Average Revenues per Transaction  Number of Transactions in Period  \
0                            244.73                               511   
1                             63.33                               147   

  Zip Code Best Selling Broad Category  \
0    50613                     whiskey   
1    50543                     whiskey   

   Average Retail (Transaction) Price per Bottle      County  County Number  \
0                                          15.29  Black Hawk              7   
1                                          16.32     Webster             94   

   Store Bottles Sold  Average Bottles Sold per Transaction  \
0                8625                                 16.88   
1                 671                                  4.56   

   Max Bottles Sold per Transaction  Min Bottles Sold per Transaction  \
0                                72                                 1   
1                                24                                 1   

   Store Volume (liters)  Average Liters Sold per Transaction  \
0                8020.35                                15.70   
1                 659.85                                 4.49   

   Store Operating Days  Number of Products Offered  \
0                    52                         244   
1                    47                          94   

   Average Transaction Revenue per Liter  Store Bottles Sold per Day  \
0                                  17.63                      165.87   
1                                  18.51                       14.28   

   Store Liters Sold per Day  Store Revenues per Day  \
0                     154.24                 2404.95   
1                      14.04                  198.09   

   Store Revenues per Bottle  Store Revenues per Liter  \
0                      14.50                     15.59   
1                      13.88                     14.11   

   Store Gross Profits per Day  Store Gross Profit Margin%  Store Unit Margin  \
0                       802.38                       33.36               4.84   
1                        66.15                       33.39               4.63   

   Store Gross Profit per Liter  Transactions per Day  
0                          5.20                  9.83  
1                          4.71                  3.13

### Combining ZIP Code Level Data to Store Data

##### Since the number of cities is similar to the number of zip codes there should be little difference in whether you group by city versus zip.  Could also group by County but that may be too blunt of a cut for our model

In [94]:
zip_stats_df = stores_2015_df.groupby(by='Zip Code', 
                                      as_index=False).agg({'Store Number': 'count',
                                                           'Store Revenues (Dollars)': np.sum,
                                                           'Store Gross Profit (Dollars)': np.sum,
                                                           'Store Bottles Sold': np.sum,
                                                           'Store Volume (liters)': np.sum
                                                          })

zip_stats_df.head()

Zip Code  Store Bottles Sold  Store Gross Profit (Dollars)  Store Number  \
0    50002                 498                       1663.48             2   
1    50003                2234                       8521.71             3   
2    50006                 508                       3017.89             1   
3    50009               16470                      72849.03             9   
4    50010               43731                     202785.51            20   

   Store Revenues (Dollars)  Store Volume (liters)  
0                   4958.04                 335.85  
1                  25459.94                1924.52  
2                   9031.81                 629.24  
3                 217957.37               14844.15  
4                 606826.07               41321.12

In [95]:
zip_stats_df.tail()

Zip Code  Store Bottles Sold  Store Gross Profit (Dollars)  Store Number  \
371    52804               27467                     102179.84            10   
372    52806               14574                      54463.01             8   
373    52807               28498                     133677.17             9   
374    56201                 276                       1362.78             1   
375    712-2                1020                       5002.98             1   

     Store Revenues (Dollars)  Store Volume (liters)  
371                 305962.90               20689.05  
372                 162914.36               11592.10  
373                 399989.92               25369.10  
374                   4086.72                 243.00  
375                  14990.99                1020.65

In [96]:
zip_stats_df = zip_stats_df.rename(columns={'Store Bottles Sold': 'Zip Bottles Sold',
                                            'Store Gross Profit (Dollars)': 'Zip Gross Profit (Dollars)',
                                            'Store Number': 'Number of Stores by Zip',
                                            'Store Revenues (Dollars)': 'Zip Revenues (Dollars)',
                                            'Store Volume (liters)': 'Zip Volume (liters)'
                                           })

zip_stats_df.head(2)

Zip Code  Zip Bottles Sold  Zip Gross Profit (Dollars)  \
0    50002               498                     1663.48   
1    50003              2234                     8521.71   

   Number of Stores by Zip  Zip Revenues (Dollars)  Zip Volume (liters)  
0                        2                 4958.04               335.85  
1                        3                25459.94              1924.52

### Now we can create Zip Code level metrics

Potential Proxies for Competiton:
- Number of Stores by Zip 
- Zip Revenues per Store
- Zip Gross Profit per Store
- Zip Gross Profit Margin% (lower competition, higher relative pricing power)

Potential Proxy for Zip Code Income Characteristics:

Zip Revenue per Bottle or Liter -- higher end products will have higher unit prices


In [97]:
zip_stats_df['Zip Revenues per Store'] = (zip_stats_df['Zip Revenues (Dollars)'] / 
                                          zip_stats_df['Number of Stores by Zip'])                                           

In [98]:
zip_stats_df['Zip Gross Profit per Store'] = (zip_stats_df['Zip Gross Profit (Dollars)'] / 
                                              zip_stats_df['Number of Stores by Zip'])           

In [99]:
zip_stats_df['Zip Gross Profit Margin%'] = (zip_stats_df['Zip Gross Profit (Dollars)'] / 
                                            zip_stats_df['Zip Revenues (Dollars)']) * 100        

In [100]:
zip_stats_df['Zip Revenue per Bottle'] = (zip_stats_df['Zip Revenues (Dollars)'] / 
                                          zip_stats_df['Zip Bottles Sold'])       

In [101]:
zip_stats_df['Zip Revenue per Liter'] = (zip_stats_df['Zip Revenues (Dollars)'] / 
                                         zip_stats_df['Zip Volume (liters)'])       

In [102]:
zip_stats_df['Zip Gross Profit per Bottle'] = (zip_stats_df['Zip Gross Profit (Dollars)'] / 
                                               zip_stats_df['Zip Bottles Sold'])       

In [103]:
zip_stats_df['Zip Gross Profit per Liter'] = (zip_stats_df['Zip Gross Profit (Dollars)'] / 
                                              zip_stats_df['Zip Volume (liters)'])       

In [104]:
zip_stats_df.head()

Zip Code  Zip Bottles Sold  Zip Gross Profit (Dollars)  \
0    50002               498                     1663.48   
1    50003              2234                     8521.71   
2    50006               508                     3017.89   
3    50009             16470                    72849.03   
4    50010             43731                   202785.51   

   Number of Stores by Zip  Zip Revenues (Dollars)  Zip Volume (liters)  \
0                        2                 4958.04               335.85   
1                        3                25459.94              1924.52   
2                        1                 9031.81               629.24   
3                        9               217957.37             14844.15   
4                       20               606826.07             41321.12   

   Zip Revenues per Store  Zip Gross Profit per Store  \
0                 2479.02                      831.74   
1                 8486.65                     2840.57   
2                 9031.81                     3017.89   
3                24217.49                     8094.34   
4                30341.30                    10139.28   

   Zip Gross Profit Margin%  Zip Revenue per Bottle  Zip Revenue per Liter  \
0                     33.55                    9.96                  14.76   
1                     33.47                   11.40                  13.23   
2                     33.41                   17.78                  14.35   
3                     33.42                   13.23                  14.68   
4                     33.42                   13.88                  14.69   

   Zip Gross Profit per Bottle  Zip Gross Profit per Liter  
0                         3.34                        4.95  
1                         3.81                        4.43  
2                         5.94                        4.80  
3                         4.42                        4.91  
4                         4.64                        4.91

In [105]:
stores_2015_df['Store Number'].nunique()

1161

In [106]:
zip_stats_df['Number of Stores by Zip'].sum() # confirm match with above to ensure groupby is working

1161

In [107]:
zip_stats_df['Zip Revenues (Dollars)'].sum() # Total Iowa Liquor Revenues in 2015

22505492.669999976

##### Merging Store Level Data and Zip Code (Market) Data

In [108]:
store_zip_df = stores_2015_df.merge(zip_stats_df, left_on='Zip Code', right_on='Zip Code', how='inner')

store_zip_df.head()

Store Number  Number of Liquor Categories Offered  \
0          2106                                   48   
1          2572                                   59   
2          2835                                   26   
3          3769                                   31   
4          3995                                   36   

   Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
0                                10.18                      41723.72   
1                                 9.96                      27681.85   
2                                10.71                       3941.47   
3                                 9.90                      17983.56   
4                                10.88                      13330.86   

   Average Transaction Gross Profit  Average Transaction Unit Margin  \
0                             81.65                             5.10   
1                             21.34                             5.00   
2                             34.27                             5.39   
3                             57.27                             4.98   
4                             69.43                             5.45   

   Transaction Gross Margin per Liter mean         City  Number of Suppliers  \
0                                     5.89  CEDAR FALLS                   32   
1                                     6.28  CEDAR FALLS                   51   
2                                     4.84  CEDAR FALLS                   21   
3                                     4.51  CEDAR FALLS                   24   
4                                     6.10  CEDAR FALLS                   24   

   Average Transaction Unit Margin%  Store Revenues (Dollars)  \
0                             33.40                 125057.54   
1                             33.47                  82799.91   
2                             33.52                  11751.32   
3                             33.47                  53725.80   
4                             33.37                  39946.95   

   Average Revenues per Transaction  Number of Transactions in Period  \
0                            244.73                               511   
1                             63.84                              1297   
2                            102.19                               115   
3                            171.10                               314   
4                            208.06                               192   

  Zip Code Best Selling Broad Category  \
0    50613                     whiskey   
1    50613                       vodka   
2    50613                       vodka   
3    50613                       vodka   
4    50613                       vodka   

   Average Retail (Transaction) Price per Bottle      County  County Number  \
0                                          15.29  Black Hawk              7   
1                                          14.96  Black Hawk              7   
2                                          16.11  Black Hawk              7   
3                                          14.87  Black Hawk              7   
4                                          16.32  Black Hawk              7   

   Store Bottles Sold  Average Bottles Sold per Transaction  \
0                8625                                 16.88   
1                5817                                  4.48   
2                 790                                  6.87   
3                3852                                 12.27   
4                2822                                 14.70   

   Max Bottles Sold per Transaction  Min Bottles Sold per Transaction  \
0                                72                                 1   
1                                60                                 1   
2                                30                                 1   
3                                66                                 6   
4                

In [109]:
store_zip_df.shape

(1161, 49)

In [110]:
store_zip_df.describe()

Store Number  Number of Liquor Categories Offered  \
count       1161.00                              1161.00   
mean        4147.58                                25.17   
std          814.66                                13.17   
min         2106.00                                 2.00   
25%         3783.00                                15.00   
50%         4364.00                                23.00   
75%         4726.00                                33.00   
max         9010.00                                62.00   

       Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
count                              1161.00                       1161.00   
mean                                  9.07                       6477.89   
std                                   1.88                      11576.09   
min                                   2.97                        176.16   
25%                                   7.85                       1323.02   
50%                                   9.05                       2771.65   
75%                                  10.12                       6602.37   
max                                  19.04                     176936.11   

       Average Transaction Gross Profit  Average Transaction Unit Margin  \
count                           1161.00                          1161.00   
mean                              36.74                             4.55   
std                               19.39                             0.94   
min                                7.96                             1.49   
25%                               25.10                             3.94   
50%                               34.03                             4.54   
75%                               44.79                             5.08   
max                              262.40                             9.54   

       Transaction Gross Margin per Liter mean  Number of Suppliers  \
count                                  1161.00              1161.00   
mean                                      5.67                19.73   
std                                       1.21                 9.36   
min                                       3.18                 1.00   
25%                                       4.96                13.00   
50%                                       5.52                18.00   
75%                                       6.21                24.00   
max                                      27.32                58.00   

       Average Transaction Unit Margin%  Store Revenues (Dollars)  \
count                           1161.00                   1161.00   
mean                              33.43                  19384.58   
std                                0.08                  34667.84   
min                               33.21                    528.24   
25%                               33.38                   3954.84   
50%                               33.42                   8266.29   
75%                               33.46                  19787.80   
max                               34.04                 530136.48   

       Average Revenues per Transaction  Number of Transactions in Period  \
count                           1161.00                           1161.00   
mean                             109.92                            177.73   
std                               58.06                            233.12   
min                               23.83                              4.00   
25%                               75.06                             40.00   
50%                              101.77                             96.00   
75%                              133.98                            195.00   
max                              787.05                           1865.00   

       Average Retail (Transaction) Price per Bottle  County Number  \
count                                        1161.00        1161.00  

### Classifying the Stores According to Sales per Day Quartile

#### What Differentiates Successful Stores from Average Stores?

Our task is to investigate the Iowa State Liquor dataset to recommend locations for our client's store expansion program.  Now that we have loaded, cleaned and parsed through the data -- it is time to synthesize some insights that will help us features for our model.

From the data we have aquired, the next logical step is to determine what are the common characteristics of 'successful' stores.

Define 'successful' stores as being in the top quartile in sales per day ~ $430

In [111]:
def make_store_quartiles(x):
    if x < 189:
        return "4th"
    elif x < 264:
        return "3rd"
    elif x < 430:
        return "2nd"
    else:
        return "1st"

In [112]:
store_zip_df['Store Revenues per Day Quartile'] = store_zip_df['Store Revenues per Day'].map(make_store_quartiles)

store_zip_df.head(5)


Store Number  Number of Liquor Categories Offered  \
0          2106                                   48   
1          2572                                   59   
2          2835                                   26   
3          3769                                   31   
4          3995                                   36   

   Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
0                                10.18                      41723.72   
1                                 9.96                      27681.85   
2                                10.71                       3941.47   
3                                 9.90                      17983.56   
4                                10.88                      13330.86   

   Average Transaction Gross Profit  Average Transaction Unit Margin  \
0                             81.65                             5.10   
1                             21.34                             5.00   
2                             34.27                             5.39   
3                             57.27                             4.98   
4                             69.43                             5.45   

   Transaction Gross Margin per Liter mean         City  Number of Suppliers  \
0                                     5.89  CEDAR FALLS                   32   
1                                     6.28  CEDAR FALLS                   51   
2                                     4.84  CEDAR FALLS                   21   
3                                     4.51  CEDAR FALLS                   24   
4                                     6.10  CEDAR FALLS                   24   

   Average Transaction Unit Margin%  Store Revenues (Dollars)  \
0                             33.40                 125057.54   
1                             33.47                  82799.91   
2                             33.52                  11751.32   
3                             33.47                  53725.80   
4                             33.37                  39946.95   

   Average Revenues per Transaction  Number of Transactions in Period  \
0                            244.73                               511   
1                             63.84                              1297   
2                            102.19                               115   
3                            171.10                               314   
4                            208.06                               192   

  Zip Code Best Selling Broad Category  \
0    50613                     whiskey   
1    50613                       vodka   
2    50613                       vodka   
3    50613                       vodka   
4    50613                       vodka   

   Average Retail (Transaction) Price per Bottle      County  County Number  \
0                                          15.29  Black Hawk              7   
1                                          14.96  Black Hawk              7   
2                                          16.11  Black Hawk              7   
3                                          14.87  Black Hawk              7   
4                                          16.32  Black Hawk              7   

   Store Bottles Sold  Average Bottles Sold per Transaction  \
0                8625                                 16.88   
1                5817                                  4.48   
2                 790                                  6.87   
3                3852                                 12.27   
4                2822                                 14.70   

   Max Bottles Sold per Transaction  Min Bottles Sold per Transaction  \
0                                72                                 1   
1                                60                                 1   
2                                30                                 1   
3                                66                                 6   
4                

##### Now that we have the stores into quartiles, we can run some groupbys and pivot tables to see what characteristics set them apart

In [113]:
store_quartiles1_df = store_zip_df.groupby(by='Store Revenues per Day Quartile', 
                                          as_index=False).agg({'Store Number': 'count',
                                                               'Store Revenues (Dollars)': [np.sum, np.mean, np.median],
                                                               'Store Gross Profit (Dollars)': [np.sum, np.mean, np.median],
                                                               'Store Bottles Sold': [np.sum, np.mean, np.median],
                                                               'Store Bottles Sold per Day': [np.sum, np.mean, np.median],
                                                               'Store Liters Sold per Day': [np.sum, np.mean, np.median],
                                                               'Store Volume (liters)': [np.sum, np.mean],
                                                               'Store Gross Profit Margin%': [np.mean, np.median],
                                                               'Store Revenues per Day': [np.mean, np.median],
                                                               'Store Revenues per Bottle': [np.mean, np.median],
                                                               'Store Gross Profits per Day': [np.mean, np.median],
                                                               'Zip Code': 'nunique',
                                                               'City': 'nunique',
                                                               'County Number': 'nunique',
                                                               'Average Bottles Sold per Transaction': np.mean,
                                                               'Average Retail (Transaction) Price per Bottle': np.mean,
                                                               'Number of Liquor Categories Offered': [np.mean, np.median],
                                                               'Number of Products Offered': [np.mean, np.median],
                                                               'Number of Transactions in Period': np.mean,
                                                               'Transactions per Day': np.mean,
                                                               'Number of Stores by Zip': np.mean,
                                                               'Zip Revenues per Store': np.mean,
                                                               'Zip Revenues (Dollars)': np.sum,
                                                               'Zip Revenue per Bottle': [np.mean, np.median],
                                                               'Number of Suppliers': np.mean,
                                                               'Store Unit Margin': [np.mean, np.median],
                                                               'Best Selling Broad Category': lambda x:x.value_counts().index[0]
                                                               
                                                              })

store_quartiles1_df.head()

Store Revenues per Day Quartile    City Store Revenues per Day          \
                                  nunique                   mean  median   
0                             1st     104                1007.00  755.39   
1                             2nd     129                 330.24  322.11   
2                             3rd     170                 222.39  219.16   
3                             4th     178                 155.69  158.31   

  Store Gross Profits per Day         Store Revenues (Dollars)            \
                         mean  median                      sum      mean   
0                      336.48  253.11                 1.62e+07  55533.97   
1                      110.39  107.71                 3.22e+06  11259.93   
2                       74.36   73.34                 1.87e+06   6418.50   
3                       52.06   52.97                 1.26e+06   4290.04   

            Store Bottles Sold                  Number of Stores by Zip  \
     median                sum     mean  median                    mean   
0  39140.35            1151182  3955.95  3087.0                    9.11   
1  11979.13             291211  1018.22   918.5                    7.63   
2   6122.65             176095   605.14   544.0                    6.57   
3   4033.20             129818   443.06   388.0                    6.08   

  Store Unit Margin        Transactions per Day Zip Code  \
               mean median                 mean  nunique   
0              4.66   4.71                 8.15      132   
1              3.99   4.08                 3.21      160   
2              3.79   3.76                 2.44      193   
3              3.41   3.45                 2.35      198   

  Zip Revenues (Dollars) Average Bottles Sold per Transaction  \
                     sum                                 mean   
0               7.04e+07                                10.21   
1               4.99e+07                                10.38   
2               4.34e+07                                 9.57   
3               3.62e+07                                 7.99   

  Number of Suppliers Store Bottles Sold per Day               Store Number  \
                 mean                        sum   mean median        count   
0               30.08                   21011.59  72.20  60.73          291   
1               18.53                    8510.71  29.76  27.76          286   
2               15.66                    6195.81  21.29  19.45          291   
3               14.67                    4814.65  16.43  15.27          293   

  Best Selling Broad Category Store Liters Sold per Day                \
                     <lambda>                       sum   mean median   
0                     whiskey                  19519.47  67.08  53.91   
1                       vodka                   6762.19  23.64  22.96   
2                     whiskey                   4916.59  16.90  16.73   
3                       vodka                   3612.32  12.33  12.30   

  Number of Liquor Categories Offered        Zip Revenues per Store  \
                                 mean median                   mean   
0                               39.58     42               27870.94   
1                               23.69     24               19458.57   
2                               19.47     18               16604.08   
3                               17.97     17               13645.42   

  Store Revenues per Bottle        Number of Products Offered         \
                       mean median                       mean median   
0                     13.94  14.10                     254.19    206   
1                     11.95  12.22                      78.14     67   
2                     11.33  11.22                      52.09     42   
3                     10.20  10.32                      45.85     39   

  Store Gross Profit Margin%        Zip Revenue per Bottle         \
                        mean median       

##### EDA Key Takeaways
- Data suggests that top performing stores owe their relative success to more to their ability to drive higher volumes, not higher pricing
- Outside of having presumeably better locations, they are able to do this by offering a much broader set of product categories 70% more than 2nd quartile and 3x the number of items (SKUs - different brands (2x by number of suppliers), different bottle sizes)
- In terms of unit volume - top quartile stores do more than 2x volume of 2nd quartile stores (in terms of bottles per day sold)
- Higher unit pricing is a factor in superior performance (but not nearly as important as volume).  Top quartile stores had average unit price 16% higher than 2Q.  Translated into 16% higher gross profits per bottle as gross profit margin % across all of the quartiles is relatively constant (perhaps due to regulation)
- Top quartile zip revenue per bottle only 4% higher than 2Q
- Gross profit margin similar across all of the quartiles - gross profit margin % may be regulated
- As the old saying goes, you cannot spend gross profit margin in percentage terms, nbut you can spend gross profit dollars
- Top performers main lever of outperformance is driving higher volumes (without knowing anything about locations), through broader product offering - which in turn probably reflects size and diversity of local market
- Even within the top quartile, performance seems to be skewed by a smaller number of 'super stores' located in a small subset of zip codes -- may need to go to street level or more granulated zip code
- Data suggests that there is room to add stores in certain high performing zips given the high gap between first quartile zip revenues per store and the other quartiles

### Try to get deeper on by looking at deciles

In [114]:
# Store Revenues per Day Deciles

print store_zip_df['Store Revenues per Day'].quantile(.90)
print store_zip_df['Store Revenues per Day'].quantile(.80)
print store_zip_df['Store Revenues per Day'].quantile(.70)
print store_zip_df['Store Revenues per Day'].quantile(.60)
print store_zip_df['Store Revenues per Day'].quantile(.50)
print store_zip_df['Store Revenues per Day'].quantile(.40)
print store_zip_df['Store Revenues per Day'].quantile(.30)
print store_zip_df['Store Revenues per Day'].quantile(.20)
print store_zip_df['Store Revenues per Day'].quantile(.10)


920.151153846
530.925576923
379.475384615
307.38
263.54175
226.59
199.367317073
176.328888889
152.307666667


In [115]:
def make_store_deciles(x):
    if x < 152:
        return 10
    elif x < 176:
        return 9
    elif x < 199:
        return 8
    elif x < 226:
        return 7
    elif x < 264:
        return 6
    elif x < 307:
        return 5
    elif x < 380:
        return 4
    elif x < 530:
        return 3
    elif x < 920:
        return 2
    else:
        return 1

In [116]:
store_zip_df['Store Revenues per Day Decile'] = store_zip_df['Store Revenues per Day'].map(make_store_deciles)

store_zip_df.head(5)

Store Number  Number of Liquor Categories Offered  \
0          2106                                   48   
1          2572                                   59   
2          2835                                   26   
3          3769                                   31   
4          3995                                   36   

   Average Transaction Cost per Bottle  Store Gross Profit (Dollars)  \
0                                10.18                      41723.72   
1                                 9.96                      27681.85   
2                                10.71                       3941.47   
3                                 9.90                      17983.56   
4                                10.88                      13330.86   

   Average Transaction Gross Profit  Average Transaction Unit Margin  \
0                             81.65                             5.10   
1                             21.34                             5.00   
2                             34.27                             5.39   
3                             57.27                             4.98   
4                             69.43                             5.45   

   Transaction Gross Margin per Liter mean         City  Number of Suppliers  \
0                                     5.89  CEDAR FALLS                   32   
1                                     6.28  CEDAR FALLS                   51   
2                                     4.84  CEDAR FALLS                   21   
3                                     4.51  CEDAR FALLS                   24   
4                                     6.10  CEDAR FALLS                   24   

   Average Transaction Unit Margin%  Store Revenues (Dollars)  \
0                             33.40                 125057.54   
1                             33.47                  82799.91   
2                             33.52                  11751.32   
3                             33.47                  53725.80   
4                             33.37                  39946.95   

   Average Revenues per Transaction  Number of Transactions in Period  \
0                            244.73                               511   
1                             63.84                              1297   
2                            102.19                               115   
3                            171.10                               314   
4                            208.06                               192   

  Zip Code Best Selling Broad Category  \
0    50613                     whiskey   
1    50613                       vodka   
2    50613                       vodka   
3    50613                       vodka   
4    50613                       vodka   

   Average Retail (Transaction) Price per Bottle      County  County Number  \
0                                          15.29  Black Hawk              7   
1                                          14.96  Black Hawk              7   
2                                          16.11  Black Hawk              7   
3                                          14.87  Black Hawk              7   
4                                          16.32  Black Hawk              7   

   Store Bottles Sold  Average Bottles Sold per Transaction  \
0                8625                                 16.88   
1                5817                                  4.48   
2                 790                                  6.87   
3                3852                                 12.27   
4                2822                                 14.70   

   Max Bottles Sold per Transaction  Min Bottles Sold per Transaction  \
0                                72                                 1   
1                                60                                 1   
2                                30                                 1   
3                                66                                 6   
4                

In [117]:
store_deciles_df = store_zip_df.groupby(by='Store Revenues per Day Decile', 
                                          as_index=False).agg({'Store Number': 'count',
                                                               'Store Revenues (Dollars)': [np.sum, np.mean, np.median],
                                                               'Store Gross Profit (Dollars)': [np.sum, np.mean, np.median],
                                                               'Store Bottles Sold': [np.sum, np.mean, np.median],
                                                               'Store Bottles Sold per Day': [np.sum, np.mean, np.median],
                                                               'Store Liters Sold per Day': [np.sum, np.mean, np.median],
                                                               'Store Volume (liters)': [np.sum, np.mean],
                                                               'Store Gross Profit Margin%': [np.mean, np.median],
                                                               'Store Revenues per Day': [np.mean, np.median],
                                                               'Store Revenues per Bottle': [np.mean, np.median],
                                                               'Store Gross Profits per Day': [np.mean, np.median],
                                                               'Zip Code': 'nunique',
                                                               'City': 'nunique',
                                                               'County Number': 'nunique',
                                                               'Average Bottles Sold per Transaction': np.mean,
                                                               'Average Retail (Transaction) Price per Bottle': np.mean,
                                                               'Number of Liquor Categories Offered': [np.mean, np.median],
                                                               'Number of Products Offered': [np.mean, np.median],
                                                               'Number of Transactions in Period': np.mean,
                                                               'Transactions per Day': np.mean,
                                                               'Number of Stores by Zip': np.mean,
                                                               'Zip Revenues per Store': np.mean,
                                                               'Zip Revenues (Dollars)': np.sum,
                                                               'Zip Revenue per Bottle': [np.mean, np.median],
                                                               'Number of Suppliers': np.mean,
                                                               'Store Unit Margin': [np.mean, np.median],
                                                               'Best Selling Broad Category': lambda x:x.value_counts().index[0]
                                                               
                                                              })

store_deciles_df

Store Revenues per Day Decile    City Store Revenues per Day           \
                                nunique                   mean   median   
0                             1      53                1599.25  1421.44   
1                             2      65                 673.11   638.77   
2                             3      73                 441.01   430.68   
3                             4      64                 339.78   338.26   
4                             5      83                 282.97   281.44   
5                             6      82                 244.85   244.07   
6                             7      91                 211.83   211.72   
7                             8      89                 188.07   188.50   
8                             9      90                 163.48   163.42   
9                            10      73                 131.97   136.30   

  Store Gross Profits per Day         Store Revenues (Dollars)            \
                         mean  median                      sum      mean   
0                      534.35  475.35                 1.10e+07  94233.45   
1                      224.91  213.35                 3.98e+06  33996.22   
2                      147.39  143.84                 2.07e+06  18303.47   
3                      113.60  113.10                 1.30e+06  11038.21   
4                       94.60   93.99                 1.01e+06   8991.57   
5                       81.87   81.70                 8.33e+05   6831.09   
6                       70.84   70.75                 7.47e+05   6494.91   
7                       62.88   62.89                 5.97e+05   5056.73   
8                       54.66   54.76                 4.90e+05   4186.06   
9                       44.13   45.53                 4.58e+05   4089.23   

            Store Bottles Sold                  Number of Stores by Zip  \
     median                sum     mean  median                    mean   
0  73476.67             741811  6340.26  5306.0                   10.16   
1  32936.63             313410  2678.72  2681.0                    8.51   
2  19840.65             173993  1539.76  1524.0                    7.70   
3  12432.84             117521   995.94   925.5                    8.28   
4   9196.35              95658   854.09   782.0                    7.15   
5   6231.52              78889   646.63   544.5                    7.11   
6   7227.66              69706   606.14   562.0                    5.68   
7   4507.77              57362   486.12   408.5                    5.98   
8   3876.58              51021   436.08   384.0                    5.71   
9   4099.59              48935   436.92   381.0                    7.12   

  Store Unit Margin        Transactions per Day Zip Code  \
               mean median                 mean  nunique   
0              4.94   4.81                12.27       75   
1              4.54   4.55                 5.96       81   
2              4.26   4.43                 4.01       84   
3              4.12   4.31                 3.07       84   
4              3.76   3.75                 3.04       95   
5              3.87   3.84                 2.51       99   
6              3.73   3.68                 2.49      102   
7              3.61   3.71                 2.25       98   
8              3.41   3.48                 2.38      102   
9              3.35   3.38                 2.34       82   

  Zip Revenues (Dollars) Average Bottles Sold per Transaction  \
                     sum                                 mean   
0               3.41e+07                                10.19   
1               2.56e+07                                10.14   
2               2.03e+07                                10.28   
3               2.29e+07                                10.86   
4               1.75e+07                                 9.99   
5               1.97e+07                                10.31   
6               1.48e+07                      

Splitting the data into deciles demonstrates that bulk of Iowa liquor sales are skewed to a relatively small amount of stores and locations.  The differences voliumes sold per day between the top and second decile are very large (2x on a bottles sold per day basis).

Clear linear relationship between the deciles and our proxies for store size (categories, vendors, items).

Impact of higher average pricing made more clear by looking at deciles.  Top decile pricing much higher than mid-table.

Higher transactions per day -- larger stores draw more foot traffic.

### Building a Regression Model

From the discussion above, it is clear that the variable we should be trying to predict is unit volume.  Given that Revenues = Volume*Price, and that from EDA it seems that higher revenue per day stores is due to their ability to drive volume, we should be trying to build a model where unit volume (in this case bottles -- could also run with liters) is the dependent variable.

Store Revenue per day and Gross profit per day all flow from Unit volume per day.  If you use them as one of your features, you would get non-sensical results.

From our EDA, see that stores with higher volume tend to:

- sell a wide range of categories - higher # of categories, vendors (proxy for brands carried), items (categories+brands+bottle sizes) -- this may be a function of a large market area served whose population have diverse tastes, ability to drive foot traffic by being a one-stop shop

- are located in a large market area -- measured by total zip code sales per store (also see if stores per zip has any impact as a proxy for competition).

- have a higher average selling price -- perhaps reflecting selling to higher income markets



#### Sort Parameters by their Correlation with 'Store Bottles Sold per Day' - dependent variable

In [118]:
store_zip_abs_correlations = abs(store_zip_df.corr()['Store Bottles Sold'])
store_zip_abs_correlations.sort_values(inplace=True, ascending=False)
store_zip_abs_correlations

Store Bottles Sold                               1.00
Store Gross Profit (Dollars)                     0.98
Store Volume (liters)                            0.98
Store Revenues (Dollars)                         0.98
Store Revenues per Day                           0.89
Store Bottles Sold per Day                       0.89
Store Gross Profits per Day                      0.89
Store Liters Sold per Day                        0.87
Number of Transactions in Period                 0.87
Number of Products Offered                       0.80
Number of Suppliers                              0.74
Transactions per Day                             0.73
Store Operating Days                             0.69
Number of Liquor Categories Offered              0.68
Store Revenues per Day Decile                    0.59
Max Bottles Sold per Transaction                 0.53
Store Number                                     0.47
Zip Gross Profit per Store                       0.45
Zip Revenues per Store      

In [119]:
##### Setting Up Features Matrix

In [120]:
from patsy import dmatrices, dmatrix, demo_data

In [121]:
y, X = dmatrices("np.log(Q('Store Bottles Sold')) ~ \
                  np.log(Q('Number of Products Offered')) + \
                  np.log(Q('Number of Stores by Zip')) + \
                  np.log(Q('Zip Revenues (Dollars)')) + \
                  np.log(Q('Average Retail (Transaction) Price per Bottle'))" \
                  , store_zip_df)

Chose to to do a log transformation on the model.  First the log transformation helps normalize the large positive skew of store volumes.  Secondly,  do it for intutive model interpretation.  if you log both your dependent (Y) and independent (X) variable(s) your regression coefficients will be elasticities and interpretation would go as follows: a 1% increase in X would lead to a ceteris paribus, B*1% increase in Y.  

In [122]:
X.shape

(1161, 5)

In [123]:
y.shape

(1161, 1)

##### Importing Regression Packages

Going to use OLS, Ridge, Lasso and Stochastic Gradient Descent models using default assumptions as a first cut.

In [124]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.preprocessing import StandardScaler

In [125]:
def fit_and_score_linear_model(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

    scaler = StandardScaler()
    scaler.fit(X_train)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    model.fit(X_train, y_train)
    
    return {'model': model, # pandas wants each row to be a dictionary
            'train_score' : model.score(X_train, y_train), 
            'test_score': model.score(X_test, y_test),
            'model_coef': model.coef_}

In [126]:
fit_and_score_linear_model(LinearRegression(), X, y) # will yield the lowest bias 

{'model': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
 'model_coef': array([[ 0.        ,  0.94407603, -0.11440242,  0.2588544 , -0.08158848]]),
 'test_score': 0.84687623844719528,
 'train_score': 0.83860512770240025}

In [127]:
fit_and_score_linear_model(Ridge(), X, y)

{'model': Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001),
 'model_coef': array([[ 0.        ,  0.94302301, -0.11207765,  0.25664765, -0.0808729 ]]),
 'test_score': 0.84698478626357709,
 'train_score': 0.83860328208749857}

In [128]:
fit_and_score_linear_model(Lasso(), X, y)

{'model': Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 'model_coef': array([ 0.,  0.,  0.,  0.,  0.]),
 'test_score': -0.00011127222003959325,
 'train_score': 0.0}

##### This Lasso Regression result is troubling -- this combined with how close the other regression results are leads me to believe something is very wrong with the way I've either set up the data, set up the data for the regression packages or in terms of feature engineering.  My instinct is that Lasso is telling me that none of the features are predictive and have been regularized down to zero.

In [129]:
fit_and_score_linear_model(SGDRegressor(), X, y)

/Users/nate_velarde/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'model': SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
        fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
        loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
        random_state=None, shuffle=True, verbose=0, warm_start=False),
 'model_coef': array([ 0.        ,  0.9507392 , -0.04332413,  0.19092371, -0.06685425]),
 'test_score': 0.84903304535883661,
 'train_score': 0.83792261263758228}

### Interpretation of results

The 3 models that yielded sensible results (OLS, Ridge and SGD) give a clear recommendation.  Locate your store in a zip code with high revenues and try an offer as broad a product selection as possible, which means you will be looking at opening a large store.  Even though high revenue ZIP codes have greater than average number of competitors, those markets seem to be large enough to more than support the large number of players.  For every 1% change in the number of products offered, you should expect 0.95% increase in store bottle sales all else equal.  Similarly, one should expect a 1% increase in the number of stores in a zip code to negatively impact sales by 0.1%.  Among high revenue zip codes, the average number of stores is ~10, so the models predict a new entrant would reduce sales volumes by 1%.  Moving into a Zip Code with higher sales will improve sales volumes by 0.20% for every 1% increase over the baseline, all else equal.  Finally, sales volume has a negative relationship with retail price.  The elasticity is very low, which makes sense as liquor sales volumes tend to be resilient in most economic environemnts and are goods that are consumers are relatively price-insensitive towards.

The 3 models above are reporting R-squared's of 83% on the test set and 84% on the training set.  They are purportedly explaining 84% of the variation in store bottle sales. Outside of the Lasso results, the R2s are reasonably high and the similarity of test to train scores suggests the model is neither overfitting or underfitting.  On the surface, this seems like a good result - a model that seems to balance bias and variance yet explains a high proportion of the data.  OLS yields the least biased model, so the failure of Ridge and SGD to meaningfully differ from OLS R2 (on both the train and test portion) suggests that the model is controlling variance as well.

However, I am a skeptic.  As mentioned above, given the Lasso results and the lack of variation between the three other models (seemingly a red flag), leads me to suspect something is amiss with either the data and/or my preparation of it for use in the regression packages.  As they say, garbage in, garbage out.



### Suggestions for Further Research

Many of the data features in this data set are very correlated to one another -- e.g., gross profit is a function of price -- therefore would get non-sensicle results if tried to use together in a model, especially one regressed on the other.  Wish had more time to supplement data set with outside data on average income levels per Zip Code or City.  It may also have been instructive to see how the data would be if it were cut by City instead of Zip Code.

I also would ahve liked to include more visual EDA, but decided to focus on trying to get the regression models up and running over charts.